In [7]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader

load_dotenv()

True

In [3]:
from services.langchain.chains import load_critisim_chain
from langchain_experimental.plan_and_execute.schema import (
    Plan,
    PlanOutputParser,
    Step,
)
import re

chain = load_critisim_chain(verbose=False)
with open("markdown/sample.md") as f:
    SAMPLE_DOCUMENT = f.read()

output = chain.run(SAMPLE_DOCUMENT)

critisims = output.split("\n\n")

class PlanningOutputParser(PlanOutputParser):
    """Planning output parser."""

    def parse(self, text: str) -> Plan:
        steps = [Step(value=v) for v in re.split("\n\s*\d+\. ", text)[1:]]
        return Plan(steps=steps)

parser = PlanningOutputParser()
plan = parser.parse(output)
plan

Plan(steps=[Step(value="**Lack of Specificity in Cybersecurity Strategy**: The policy outlines the responsibilities of the Cybersecurity Committee, including reviewing and providing oversight on various aspects of the company's cybersecurity posture. However, it does not provide specific details about the cybersecurity strategy, expectations, or measures that will be used to ensure the effectiveness of the cybersecurity program."), Step(value="**Absence of Employee Awareness & Training Program**: The policy does not provide information about the company's approach to cybersecurity awareness and training for employees. Employee behavior is one of the most significant factors in many cybersecurity incidents and breaches, and an effective cybersecurity policy should outline how the company plans to manage this risk."), Step(value='**Lack of Clear Communication Channels**: The policy does not specify the communication channels that will be used to escalate potential security issues or brea

In [5]:
import json

with open("test.json", "r") as f:
    data = json.loads(f.read())
    print(list(data.keys()))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [4]:
from services.langchain.chains import load_critisim_planner, load_redrafting_chain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4")
planner = load_critisim_planner(llm=llm)
drafter = load_redrafting_chain(llm=llm)

with open("markdown/sample.md") as f:
    SAMPLE_DOCUMENT = f.read()

plan = planner.plan({ "input": SAMPLE_DOCUMENT })

for step in plan.steps:
    print(step.value)
    output = drafter({ "critisim": step.value, "context": SAMPLE_DOCUMENT })


Plan(steps=[Step(value="**Employee Training**: The policy doesn't explicitly mention the training of employees in cybersecurity awareness and best practices. Since human error is often a significant cybersecurity risk, regular training for all employees is critical."), Step(value='**Third-Party Risk Management**: The policy mentions third-party audits but does not clearly address the management of cybersecurity risks associated with third-party vendors or service providers. A comprehensive policy should include procedures for assessing and mitigating risks from third parties.'), Step(value='**Regular Policy Update**: The policy does not explicitly state how often it will be reviewed and updated. Given the rapidly changing nature of cybersecurity threats, it is advisable to review and update the policy at least annually or whenever there are significant changes in the company’s IT environment.'), Step(value='**Incident Reporting**: The policy lacks a clear process for reporting cybersec